# OpenACC 指令

此实验面向 Fortran 程序员。如果您更喜欢使用 C/C++，请单击 [此链接](../C/README.ipynb)

不要忘记查看其他 [OpenACC 资源](https://www.openacc.org/resources) 并加入我们的 [OpenACC Slack 频道](https://www.openacc.org/community#slack)，以分享您的经验并从社区获得更多帮助。

---

让我们执行下面的单元格以显示有关服务器上运行的 GPU 的信息。为此，请执行下面的单元格块，方法是将焦点放在它上面（用鼠标单击它），然后按 Ctrl-Enter，或按上面工具栏中的播放按钮。如果一切顺利，您应该会看到灰色单元格下方返回一些输出。

In [ ]:
!nvidia-smi

---

## 简介

我们这个实验的目标是了解代码分析到底是什么，以及如何使用它来帮助我们编写强大的并行程序。

![development_cycle.png](../images/development_cycle.png)

这是 OpenACC 3 步开发周期。

**分析**您的代码，并预测哪里可以发现潜在的并行性。使用分析器来帮助了解代码中正在发生的事情以及可能存在并行性的地方。

**并行化**您的代码，从最耗时的部分开始。专注于维护程序的正确结果。

**优化**您的代码，专注于最大化性能。在早期并行化期间，性能可能不会一次性提高。

我们目前正在处理**分析**步骤。在进入接下来的两个步骤之前，我们将使用 NVIDIA Nsight Systems 分析器来了解一个相对简单的示例代码。

---

## 运行代码

分析此代码的第一步是运行它。我们需要在进行任何更改之前记录程序的结果，以便稍后将其与并行代码的结果进行比较。记录程序运行所需的时间也很重要，因为这将是我们判断并行化是否能提高性能的主要指标。

In [ ]:
!nvfortran -fast -o laplace laplace2d.f90 jacobi.f90 && echo "Compilation Successful!" && ./laplace

### 可选：查看代码

如果您想复习一下我们正在处理的代码文件，您可以使用下面的两个链接查看它们。

[jacobi.f90](jacobi.f90)

[laplace2d.f90](laplace2d.f90)

### 可选：分析代码

如果您想使用 Nsight Systems 分析您的代码，请按照 **[Lab2](../../../module2/assets/Fortran/README.ipynb)** 中的说明进行操作，并将 NVTX 添加到您的代码中以手动检测应用程序。

---

## OpenACC 指令

OpenACC 方法使用指令来修改程序的操作。指令是我们可以添加到代码中的注释，它向编译器建议我们想要发生的事情（例如，并行化循环），而无需我们对源代码中的底层算法进行任何更改。这是一种非常强大的方法，因为它允许我们编写一次代码，该代码可以在有或没有 OpenACC 支持的多个平台上运行，因为指令可以作为注释被忽略，这意味着我们有一个可以正确顺序和并行运行的单一源代码。

### OpenACC 语法

`!$acc <directive> <clauses>`

`!$acc` 在 Fortran 中就是所谓的“编译器指令”。它们与程序员注释非常相似，因为该行以注释语句 `!` 开头。注释之后是 `$acc`。具有适当命令行选项的 OpenACC 兼容编译器可以将其解释为“指导”编译器的 OpenACC 指令，而不会破坏代码。如果编译器不理解 `!$acc`，它可以忽略它，而不是抛出语法错误，因为它只是一个注释。

**directives** 是 OpenACC 中的命令，它将告诉编译器执行某些操作。目前，我们将仅使用允许编译器并行化我们的代码的指令。

**clauses*** 是对我们的指令的添加/更改。这些包括（但不限于）优化。我更喜欢这样想：指令描述了我们的编译器要执行的一般操作（例如，并行化我们的代码），而子句允许程序员更具体（例如，我们具体希望如何并行化代码）。


---

### 并行指令

我们将在本实验中介绍三个指令：并行、内核和循环。一旦我们理解了这三个指令，您将需要使用您喜欢的指令并行化我们的 laplace 代码（或者如果您愿意，可以使用所有指令！）

*并行指令*可能是最直接的指令。它将标记代码的并行化区域（这通常仅包括并行化单个 for 循环。）让我们来看看：

```fortran
    !$acc parallel loop
    do i=1,N
        < loop code >
    enddo
```

我们也可以定义一个“并行区域”。并行区域可以有多个循环（虽然通常不建议这样做！）并行区域是最外层循环内包含的所有内容。

```fortran
!$acc parallel
    !$acc loop
    do i=1,N
        < loop code >
    enddo

```

`!$acc parallel loop` 将标记下一个循环以进行并行化。包含 **`loop`** 非常重要，否则您将无法正确并行化循环。*parallel 指令* 告诉编译器“冗余并行化”代码。`loop` 指令明确告诉编译器我们希望并行化循环。让我们看一个例子来了解为什么 *loop 指令* 如此重要。

![parallel1](../images/parallel1f.png)
![parallel2](../images/parallel2f.png)
![parallel3](../images/parallel3f.png)

我们很快将转到下一个指令（内核指令），它也允许我们并行化代码。我们还将标记这两个指令之间的差异。话虽如此，以下信息对于 *并行指令* 来说是完全独特的：

*并行指令* 将许多决定留给程序员。程序员将决定什么是可并行的，什么不是可并行的。程序员还必须提供所有优化 - 编译器不承担任何责任。如果在并行化代码时发生任何错误，程序员将负责识别并纠正它们。

我们很快就会看到内核指令在所有这些方面是如何完全相反的。

### 可选：使用并行指令并行化代码

建议您在更改拉 laplace 代码之前学习所有三个指令。但是，如果您希望*立即*尝试*并行指令*，则可以使用以下链接编辑 laplace 代码。

[jacobi.f90](jacobi.f90)

[laplace2d.f90](laplace2d.f90)

（请务必按 ctrl+s 保存所做的更改）

您可以通过运行以下脚本来运行您的代码：

In [ ]:
!nvfortran -fast -ta=multicore -Minfo=accel -o laplace_parallel laplace2d.f90 jacobi.f90 && ./laplace_parallel

---

### Kernels 指令

kernels 指令允许程序员后退，完全依赖编译器。让我们看看语法：

```fortran
!$acc kernels
do i=1,N
    < loop code >
enddo
!$acc end kernels
```

就像在 `parallel` 指令示例中一样，我们正在并行化单个循环。回想一下，使用 `parallel` 指令时，它必须始终与 `loop` 指令配对，否则代码将无法正确并行化。`kernels` 指令不遵循相同的规则，在某些编译器中，添加 *loop 指令* 可能会限制编译器优化代码的能力。

在这种情况下，您还需要包含语句 `!$acc end kernels` ，以便编译器知道指令的“范围”。

如前所述，`kernels` 指令与 `parallel` 指令完全相反。这意味着编译器正在做出许多假设，甚至可能推翻程序员并行化代码的决定。此外，默认情况下，编译器将尝试优化循环。编译器通常非常擅长优化循环，有时可能能够以程序员无法描述的方式优化循环。但是，通常情况下，程序员可以通过自己优化循环来实现更好的性能。

如果您遇到编译器拒绝并行化循环的情况，您可以推翻编译器的决定。 （但是，请记住，通过覆盖编译器的决定，您要对并行化代码时发生的任何错误负责！）在此代码段中，我们使用独立子句来向编译器保证我们认为该循环是可并行的。

```fortran
!$acc kernels loop independent
do i=1,N
    < loop code >
enddo
!$acc end kernels
```

`kernels` 指令的最大优势之一是它能够同时并行化多个循环。例如，在下面的代码段中，我们能够通过利用 kernels 区域（类似于我们之前看到的并行区域）有效地同时并行化两个循环。这是通过将语句 `!$acc end kernels` 放在指令区域的末尾来实现的。

```fortran
!$acc kernels
do i=1,N
   < loop code >
enddo
    
< some other sequential code >
    
do j=1,M
   < loop code >
enddo
!$acc end kernels
```

通过使用 kernels 指令，我们可以并行化多个循环（实际上，我们可以并行化任意数量的循环）。我们还可以在循环之间包含顺序代码，而无需包含多个指令。与之前类似，让我们看一个 kernels 指令如何工作的可视化示例。

![kernels1](../images/kernels1f.png)
![kernels2](../images/kernels2f.png)

在转到最后一个指令（*loop 指令*）之前，让我们回顾一下是什么让 parallel 和 kernels 指令在功能上如此不同。

*parallel 指令* 为程序员提供了大量控制权。程序员决定并行化什么以及如何并行化。并行化过程中的任何错误都是程序员的错。建议对要并行化的每个循环使用 *parallel 指令*。

*kernel 指令* 将大部分控制权留给编译器。编译器将分析循环，并决定并行化哪些循环。它可能会拒绝并行化某些循环，但程序员可以推翻这一决定。您可以使用 kernels 指令来并行化大段代码，这些部分可能包含多个循环。

### 可选：使用 Kernels 指令并行化代码

建议您在更改 laplace 代码之前学习所有三个指令。但是，如果您希望*立即*试用 kernels 指令，则可以使用以下链接编辑 laplace 代码。密切关注编译器反馈，并准备将*独立*子句添加到您的循环中。

[jacobi.f90](jacobi.f90)

[laplace2d.f90](laplace2d.f90)

（请务必按 ctrl+s 保存所做的更改）

您可以通过运行以下脚本来运行您的代码：

In [ ]:
!nvfortran -fast -ta=multicore -Minfo=accel -o laplace_parallel laplace2d.f90 jacobi.f90 && ./laplace_parallel

---

### 循环指令

我们已经看到 *循环指令* 被使用和提及了几次；现在是时候正式定义它了。 *循环指令* 有两个主要用途：
* 标记单个循环以进行并行化
* 允许我们明确定义循环的优化/更改

循环优化是另一个实验室的主题，因此现在我们将重点关注并行化方面。为了使 *循环指令* 正常工作，它必须包含在并行或内核指令中。

例如：

```fortran
!$acc parallel loop
do i=1,N
    < loop code >
enddo
```

或

```fortran
!$acc kernels
do i=1,N
   < loop code >
enddo
!$acc end kernels
```

使用 `parallel` 指令时，必须包含 *loop 指令* 才能使代码正常运行。使用 `kernels` 指令时，*loop 指令* 是隐含的，不需要包含。

我们还可以使用 *loop 指令* 来并行化多维循环嵌套。根据您使用的并行硬件，您可能无法实现多循环并行。某些并行硬件的并行能力有限，因此并行化内部循环不会提供任何额外的性能（尽管也不会损害程序）。在本实验中，我们使用多核 CPU 作为并行硬件，因此多循环并行化并不完全可行。但是，当使用 GPU（我们将在下一个实验中使用！）时，我们可以利用多循环并行。

无论哪种方式，多循环并行都是这样的：

```fortran
!$acc parallel loop
do i=1,N
    !$acc loop
    do j=1,M
        < loop code >
    enddo
enddo
```

`kernels` 指令在并行化嵌套循环方面也非常出色。我们可以使用 `kernels` 指令重新创建上述相同的代码：

```fortran
!$acc kernels
do i=1,N
   do j=1,M
        < loop code >
    enddo
enddo
!$acc end kernels
```

请注意，就像以前一样，我们不需要包含*loop 指令*。

## 并行化 laplace 代码

利用您对并行、内核和 *loop 指令* 的了解，将 OpenACC 指令添加到我们的 laplace 代码并对其进行并行化。您可以通过选择以下链接来编辑代码：

[jacobi.f90](jacobi.f90)

[laplace2d.f90](laplace2d.f90)

（请务必按 ctrl+s 保存所做的更改）

---

要在多核 CPU 上编译并运行并行代码，请运行以下命令：

In [ ]:
!nvfortran -fast -ta=multicore -Minfo=accel -o laplace_parallel laplace2d.f90 jacobi.f90 && ./laplace_parallel

---

如果您在任何时候觉得自己犯了错误，并希望将代码重置为原来的状态，您可以运行以下命令：

In [ ]:
!cp ./solutions/sequential/jacobi.f90 ./jacobi.f90 && cp ./solutions/sequential/laplace2d.f90 ./laplace2d.f90 && echo "Reset Complete"

---

如果您想在任何时候重新运行顺序代码来检查结果/性能，您可以运行以下命令：

In [ ]:
!cd solutions/sequential && nvfortran -fast -o laplace_seq laplace2d.f90 jacobi.f90 && ./laplace_seq

---

如果您想查看有关我们正在运行的 CPU 的信息，您可以运行以下命令：

In [ ]:
!pgcpuid

### 可选：编译多核代码

完成本实验不需要了解如何编译多核代码。但是，如果您以后想要并行化自己的个人代码，它将很有用。

**-Minfo** 将向我们提供有关代码优化和限制的编译器反馈。
**-Minfo=accel** 将仅向我们提供有关我们的 OpenACC 并行化/优化的反馈。
**-Minfo=all** 将为我们提供所有可能的反馈，包括我们的并行化/优化、顺序代码优化和顺序代码限制。
**-ta** 将允许我们为特定目标并行硬件编译代码。如果没有此标志，代码将被编译为顺序执行。
**-ta=multicore** 将允许我们为多核 CPU 编译代码。

### 可选：分析多核代码

如果您想使用 Nsight Systems 分析您的代码，请按照 **[Lab2](../../../module2/assets/Fortran/README.ipynb)** 中的说明进行操作，并将 NVTX 添加到您的代码中以手动检测应用程序。

---

## 总结

如果您想检查结果，请运行以下脚本。

In [ ]:
!cd solutions/multicore && nvfortran -fast -ta=multicore -Minfo=accel -o laplace_parallel laplace2d.f90 jacobi.f90 && ./laplace_parallel

如果您想查看解决方案代码，可以使用以下链接。

**使用并行指令**

[jacobi.f90](solutions/multicore/jacobi.f90)

[laplace2d.f90](solutions/multicore/laplace2d.f90)

**使用内核指令**

[jacobi.f90](solutions/multicore/kernels/jacobi.f90)

[laplace2d.f90](solutions/multicore/kernels/laplace2d.f90)

我们能够使用 **parallel** 或 **kernels** 指令将我们的代码并行化用于少数不同的硬件。我们还能够通过使用 parallel/kernels 指令内的 **loop** 指令来定义额外的并行级别。您还可以使用这些指令来并行化嵌套循环。

此时，我们可以对代码进行一些优化，但在大多数情况下，我们的多核代码不会变得更快。在下一个实验中，我们将把注意力转移到 GPU 加速器的编程上，在学习 GPU 的同时，我们将讨论如何在 OpenACC 中处理内存管理。

---

## 额外任务

1. 如果您选择仅使用其中一个指令（并行或内核），则返回并使用另一个指令。比较两个版本的运行时，并对两者进行分析。

2. 如果您想要一些关于使用 OpenACC 并行化代码的额外课程，OpenACC YouTube 页面上有一个 OpenACC 简介视频系列。该系列的前两个视频涵盖了本实验中涵盖的很多内容。

[OpenACC 并行编程简介 - 第 1 部分](https://youtu.be/PxmvTsrCTZg)

[OpenACC 并行编程简介 - 第 2 部分](https://youtu.be/xSCD4-GV41M)

3. 如前所述，多核加速器只能利用一个级别的并行性。但是，GPU 可以利用更多。确保使用您在实验的 **循环指令** 部分学到的技能，并并行化代码中的多维循环。然后运行下面的脚本以在 GPU 上运行代码。将结果（包括编译器反馈）与我们的多核实现进行比较。

In [ ]:
!nvfortran -fast -ta=tesla:managed -Minfo=accel -o laplace_gpu laplace2d.f90 jacobi.f90 && ./laplace_gpu

---

## 实验后总结

如果您想下载此实验以供以后查看，建议您转到浏览器的文件菜单（而不是 Jupyter 笔记本文件菜单）并保存完整的网页。这将确保图像也被复制下来。

您还可以执行以下单元块来创建您正在处理的文件的 zip 文件，并使用下面的链接下载它。

In [ ]:
%%bash
rm -f openacc_files.zip
zip -r openacc_files.zip *

**执行上述 zip 命令后**，您应该能够通过按住 <mark>Shift</mark> 并<mark>右键单击</mark> [此处](openacc_files.zip) 并选择*将链接另存为*来下载并保存 zip 文件。

# 许可

本材料由 NVIDIA Corporation 根据知识共享署名 4.0 国际 (CC BY 4.0) 发布。